In [1]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
from openai import OpenAI
import streamlit as st
import spotipy 

# Cargar variables de entorno desde un archivo .env
load_dotenv()

# Importar funciones de gestión
from utils.chat_gpt_manager import (
    get_chords_chat,
    create_song
)
from utils.spotify_manager import (
    get_similar_songs,
    get_audio_analysis,
    get_audio_features,
    get_song_and_artist
)
from utils.functions import (
    add_info_list_df,
    convert_chords_with_minor,
    standardize_chord_symbols,
    transpose_chords_to_key,
    check_track_csv,
    filter_existing_vector_id,
    to_embbed_text_column,
    expand_and_remove_original_columns,
    drop_columns_with_many_nulls,
    get_user_input,
    handle_input_search,
    calc_diff,
    export_unique_rows,
    create_metadata_column,
    clean_metadata_column,
    insert_into_pinecone,
    create_dataframe_for_pinecone,
    realizar_consulta

    
)

from utils.pinecone_manager import (
    initialize_pinecone, 
    setup_index
)

from utils.lyrics_manager import insert_lyrics_db

from pinecone import Pinecone, ServerlessSpec
from pinecone.grpc import PineconeGRPC as Pinecone

# Initizalize pinecone

In [2]:
pc = initialize_pinecone()

In [3]:
index = setup_index(pc)

Index 'song-starter-index' already exists, skipping creation.


In [4]:
index_name= pc.list_indexes().indexes[0].name

# API de Spotify

Aquí se utiliza la API de Spotify para obtener recomendaciones de canciones basadas en una canción dada. 

Input:

In [5]:
input_song, input_artist = get_user_input()

In [6]:
# Llamar a la función para obtener el track ID
input_track_id=handle_input_search(input_song, input_artist)

In [7]:
input_song , input_artist = get_song_and_artist(input_track_id)

In [8]:
input_data = pd.DataFrame({'id': [input_track_id]})
input_data['name'] = input_song
input_data['artists'] = input_artist

In [9]:
input_data

,id,name,artists
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira


Similar songs

In [10]:
tracks_data = get_similar_songs(input_track_id, 5)

In [11]:
for track in tracks_data:
    print(track['name'])


Un Poco de Amor
La Fiesta
Hoy Tengo Ganas De Ti
Amigos (con Maria Becerra)
No Pasa Nada


In [12]:
tracks_data = pd.DataFrame(tracks_data)

In [13]:
tracks_data.head(2)

,id,name,artists
0,6dwhYg12fOFw8aUrnCOOqC,Un Poco de Amor,Shakira
1,2o6pi3nx1UAYxuBccabCMQ,La Fiesta,Pedro Capó


MIRAR ESTO, cómo evitar que a partir de aquí todo de error si ya están todas en pinecone

In [14]:
filter_existing_vector_id(index, tracks_data)            

In [15]:

tracks_data.reset_index(drop=True, inplace=True)

In [16]:
tracks_data

,id,name,artists
0,6dwhYg12fOFw8aUrnCOOqC,Un Poco de Amor,Shakira
1,20NeR3CwuyGfRktTncKRU7,Amigos (con Maria Becerra),"Pablo Alborán, Maria Becerra"
2,6Gusd5o0sQD1jmmoQlvNw4,No Pasa Nada,Ha*Ash


In [17]:
csv= pd.read_csv('mi_archivo_temporal.csv')

In [18]:
if len(tracks_data)>0:
    df=check_track_csv(tracks_data, csv)

In [19]:
if len(df)>0:    
    df.reset_index(drop=True, inplace=True)

# API de ChatGPT

Aquí se utiliza la API de Chat GPT, para obtener los acordes e instrumentos de las canciones

Input:

In [20]:
add_info_list_df(input_data, get_chords_chat, 'chords', max_retries=3)

,id,name,artists,chords
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[C, G, Am, F, C, G, F, F]"


In [21]:
add_info_list_df(input_data, get_chords_chat, 'instruments', max_retries=3)

,id,name,artists,chords,instruments
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[C, G, Am, F, C, G, F, F]","[D, A, Bm, G, D, A, G, A]"


Related tracks:

In [22]:
add_info_list_df(df, get_chords_chat, 'chords', max_retries=3)

,id,name,artists,chords
0,6dwhYg12fOFw8aUrnCOOqC,Un Poco de Amor,Shakira,"[C, G, Am, F, G, D, Em, C]"
1,20NeR3CwuyGfRktTncKRU7,Amigos (con Maria Becerra),"Pablo Alborán, Maria Becerra","[C, G, Am, F, C, Em, F, G]"
2,6Gusd5o0sQD1jmmoQlvNw4,No Pasa Nada,Ha*Ash,"[C, G, Am, F, G, D, Em, C]"


In [23]:
add_info_list_df(df, get_chords_chat, 'instruments', max_retries=3)

,id,name,artists,chords,instruments
0,6dwhYg12fOFw8aUrnCOOqC,Un Poco de Amor,Shakira,"[C, G, Am, F, G, D, Em, C]","[C, G, Am, F, C, Em, F, G]"
1,20NeR3CwuyGfRktTncKRU7,Amigos (con Maria Becerra),"Pablo Alborán, Maria Becerra","[C, G, Am, F, C, Em, F, G]","[C, G, Am, F, C, G, Am, F]"
2,6Gusd5o0sQD1jmmoQlvNw4,No Pasa Nada,Ha*Ash,"[C, G, Am, F, G, D, Em, C]","[C, G, Am, F, D, A, Bm, G]"


# API de Spotify para nuevas columnas

Input:

In [24]:
get_audio_features(input_data)

,id,name,artists,chords,instruments,speechiness,instrumentalness
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[C, G, Am, F, C, G, F, F]","[D, A, Bm, G, D, A, G, A]",0.0337,0


In [25]:
get_audio_analysis(input_data)

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[C, G, Am, F, C, G, F, F]","[D, A, Bm, G, D, A, G, A]",0.0337,0,1,0,108.977,-4.767


In [26]:
insert_lyrics_db(input_data)

(                       id                         name  artists  \
 0  3TY1PFZXmYeiLcXygB74My  Suerte (Whenever, Wherever)  Shakira   
 
                       chords                instruments  speechiness  \
 0  [C, G, Am, F, C, G, F, F]  [D, A, Bm, G, D, A, G, A]       0.0337   
 
    instrumentalness  key  mode    tempo  loudness  \
 0                 0    1     0  108.977    -4.767   
 
                                                theme  
 0  Suerte que en el sur hayas nacido\nY que burle...  ,
 'Suerte que en el sur hayas nacido\nY que burlemos las distancias,\nSuerte que es haberte conocido\nY por ti amar tierras extrañas\nYo puedo escalar los andes solo\nPor ir a contar tus lunares,\n\nContigo celebro y sufro todo')

In [27]:
input_data['chords'].dropna(inplace=True)
input_data = input_data[input_data['chords'].apply(lambda x: x != [])]

In [28]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme
0,3TY1PFZXmYeiLcXygB74My,"Suerte (Whenever, Wherever)",Shakira,"[C, G, Am, F, C, G, F, F]","[D, A, Bm, G, D, A, G, A]",0.0337,0,1,0,108.977,-4.767,Suerte que en el sur hayas nacido\nY que burle...


Related tracks:

In [29]:
get_audio_features(df)

,id,name,artists,chords,instruments,speechiness,instrumentalness
0,6dwhYg12fOFw8aUrnCOOqC,Un Poco de Amor,Shakira,"[C, G, Am, F, G, D, Em, C]","[C, G, Am, F, C, Em, F, G]",0.0412,0.000007
1,20NeR3CwuyGfRktTncKRU7,Amigos (con Maria Becerra),"Pablo Alborán, Maria Becerra","[C, G, Am, F, C, Em, F, G]","[C, G, Am, F, C, G, Am, F]",0.0450,0.000000
2,6Gusd5o0sQD1jmmoQlvNw4,No Pasa Nada,Ha*Ash,"[C, G, Am, F, G, D, Em, C]","[C, G, Am, F, D, A, Bm, G]",0.0306,0.000000


In [30]:
get_audio_analysis(df)

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness
0,6dwhYg12fOFw8aUrnCOOqC,Un Poco de Amor,Shakira,"[C, G, Am, F, G, D, Em, C]","[C, G, Am, F, C, Em, F, G]",0.0412,0.000007,11,1,101.451,-9.947
1,20NeR3CwuyGfRktTncKRU7,Amigos (con Maria Becerra),"Pablo Alborán, Maria Becerra","[C, G, Am, F, C, Em, F, G]","[C, G, Am, F, C, G, Am, F]",0.0450,0.000000,11,0,157.002,-3.459
2,6Gusd5o0sQD1jmmoQlvNw4,No Pasa Nada,Ha*Ash,"[C, G, Am, F, G, D, Em, C]","[C, G, Am, F, D, A, Bm, G]",0.0306,0.000000,0,1,136.145,-5.068


In [ ]:
insert_lyrics_db(df)

Timeout o fallo! Usando GPT para obtener el tema para Pablo Alborán, Maria Becerra - Amigos (con Maria Becerra)


In [129]:
df_vectorial= df.copy()
df_vectorial['chords'].dropna(inplace=True)
df_vectorial = df_vectorial[df_vectorial['chords'].apply(lambda x: x != [])]

# Separate columns

Input:

In [130]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme
0,42VgiRyg0YjtRVLiPFNxPJ,Zapatillas,El Canto Del Loco,"[C, G, Am, F, C, G, Am, F]","[C, G, Am, F, C, G, F, G]",0.178,0,8,0,198.076,-2.382,Estoy cansado de salir de noche y ver siempre ...


In [131]:
input_data['chords_1'] = input_data['chords'].apply(lambda x: x[:4]) 
input_data['chords_2'] = input_data['chords'].apply(lambda x: x[4:])

In [132]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2
0,42VgiRyg0YjtRVLiPFNxPJ,Zapatillas,El Canto Del Loco,"[C, G, Am, F, C, G, Am, F]","[C, G, Am, F, C, G, F, G]",0.178,0,8,0,198.076,-2.382,Estoy cansado de salir de noche y ver siempre ...,"[C, G, Am, F]","[C, G, Am, F]"


In [133]:
input_data['chords_1'] = input_data['chords_1'].apply(standardize_chord_symbols)
input_data['chords_2'] = input_data['chords_2'].apply(standardize_chord_symbols)

In [134]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2
0,42VgiRyg0YjtRVLiPFNxPJ,Zapatillas,El Canto Del Loco,"[C, G, Am, F, C, G, Am, F]","[C, G, Am, F, C, G, F, G]",0.178,0,8,0,198.076,-2.382,Estoy cansado de salir de noche y ver siempre ...,"[C, G, Am, F]","[C, G, Am, F]"


In [135]:
input_data['chords_1_numeric'] = input_data['chords_1'].apply(convert_chords_with_minor)
input_data['chords_2_numeric'] = input_data['chords_2'].apply(convert_chords_with_minor)

In [136]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric
0,42VgiRyg0YjtRVLiPFNxPJ,Zapatillas,El Canto Del Loco,"[C, G, Am, F, C, G, Am, F]","[C, G, Am, F, C, G, F, G]",0.178,0,8,0,198.076,-2.382,Estoy cansado de salir de noche y ver siempre ...,"[C, G, Am, F]","[C, G, Am, F]","[0, 7, 21, 5]","[0, 7, 21, 5]"


In [137]:
col_chords = ['chords_1_numeric', 'chords_2_numeric']

# Aplicar la función a cada fila
for col in col_chords:
    input_data[f'normal_{col}'] = input_data.apply(lambda row: transpose_chords_to_key(row, col), axis=1)

input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric
0,42VgiRyg0YjtRVLiPFNxPJ,Zapatillas,El Canto Del Loco,"[C, G, Am, F, C, G, Am, F]","[C, G, Am, F, C, G, F, G]",0.178,0,8,0,198.076,-2.382,Estoy cansado de salir de noche y ver siempre ...,"[C, G, Am, F]","[C, G, Am, F]","[0, 7, 21, 5]","[0, 7, 21, 5]","[-8, -1, 13, -3]","[-8, -1, 13, -3]"


In [138]:
for col in col_chords:
    input_data[f'diff_{col}'] = input_data.apply(lambda row: calc_diff(row[col]), axis=1)

input_data


,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,42VgiRyg0YjtRVLiPFNxPJ,Zapatillas,El Canto Del Loco,"[C, G, Am, F, C, G, Am, F]","[C, G, Am, F, C, G, F, G]",0.178,0,8,0,198.076,-2.382,Estoy cansado de salir de noche y ver siempre ...,"[C, G, Am, F]","[C, G, Am, F]","[0, 7, 21, 5]","[0, 7, 21, 5]","[-8, -1, 13, -3]","[-8, -1, 13, -3]","[7, 14, -16]","[7, 14, -16]"


In [139]:
input_data.replace('\n', ' ', regex=True, inplace=True)
#elimino los saltos de línea en la letra y temática de la canción

In [140]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,42VgiRyg0YjtRVLiPFNxPJ,Zapatillas,El Canto Del Loco,"[C, G, Am, F, C, G, Am, F]","[C, G, Am, F, C, G, F, G]",0.178,0,8,0,198.076,-2.382,Estoy cansado de salir de noche y ver siempre ...,"[C, G, Am, F]","[C, G, Am, F]","[0, 7, 21, 5]","[0, 7, 21, 5]","[-8, -1, 13, -3]","[-8, -1, 13, -3]","[7, 14, -16]","[7, 14, -16]"


Related tracks:

In [141]:
df_vectorial['chords_1'] = df_vectorial['chords'].apply(lambda x: x[:4]) 
df_vectorial['chords_2'] = df_vectorial['chords'].apply(lambda x: x[4:])
df_vectorial.drop(columns=['chords'], inplace=True)

In [142]:
df_vectorial.head(3)

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2
0,1uJnb0kS96zFOrjCI97JV8,He Vuelto,El Barrio,"[C, G, Am, F, F, G, C, Am]",0.1760,0.000000,4,1,164.974,-3.762,He vuelto\nSe acabó este descansito\nAhora toc...,"[Am, G, F, E]","[C, G, Am, E]"
1,1kG7gfH4HDUlDht0VZYkiP,Libertad,Nil Moliner,"[C, G, Am, F, G, D, Em, C]",0.0303,0.000000,4,1,132.081,-6.707,Como un tsunami siento el aire entre mis dedos...,"[C, G, Am, F]","[C, F, G, C]"
2,2yptV8t4OH935km6IZWt1l,Cosas Que Suenan A...,Maldita Nerea,"[C, G, Am, F, F, G, C, Am]",0.0350,0.000019,2,1,148.968,-8.005,Yo nado en mares pequeños\nY que todo salga bi...,"[C, G, Am, F]","[F, C, G, Am]"


In [143]:
# Nos ocupamos de bemoles o sostenidos para estandarizarlos
df_vectorial['chords_1'] = df_vectorial['chords_1'].apply(standardize_chord_symbols)
df_vectorial['chords_2'] = df_vectorial['chords_2'].apply(standardize_chord_symbols)

In [144]:
df_vectorial.head(3)

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2
0,1uJnb0kS96zFOrjCI97JV8,He Vuelto,El Barrio,"[C, G, Am, F, F, G, C, Am]",0.1760,0.000000,4,1,164.974,-3.762,He vuelto\nSe acabó este descansito\nAhora toc...,"[Am, G, F, E]","[C, G, Am, E]"
1,1kG7gfH4HDUlDht0VZYkiP,Libertad,Nil Moliner,"[C, G, Am, F, G, D, Em, C]",0.0303,0.000000,4,1,132.081,-6.707,Como un tsunami siento el aire entre mis dedos...,"[C, G, Am, F]","[C, F, G, C]"
2,2yptV8t4OH935km6IZWt1l,Cosas Que Suenan A...,Maldita Nerea,"[C, G, Am, F, F, G, C, Am]",0.0350,0.000019,2,1,148.968,-8.005,Yo nado en mares pequeños\nY que todo salga bi...,"[C, G, Am, F]","[F, C, G, Am]"


In [145]:
# Convertimos los acordes a valores numéricos
df_vectorial['chords_1_numeric'] = df_vectorial['chords_1'].apply(convert_chords_with_minor)
df_vectorial['chords_2_numeric'] = df_vectorial['chords_2'].apply(convert_chords_with_minor)

In [146]:
df_vectorial.head(3)

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric
0,1uJnb0kS96zFOrjCI97JV8,He Vuelto,El Barrio,"[C, G, Am, F, F, G, C, Am]",0.1760,0.000000,4,1,164.974,-3.762,He vuelto\nSe acabó este descansito\nAhora toc...,"[Am, G, F, E]","[C, G, Am, E]","[21, 7, 5, 4]","[0, 7, 21, 4]"
1,1kG7gfH4HDUlDht0VZYkiP,Libertad,Nil Moliner,"[C, G, Am, F, G, D, Em, C]",0.0303,0.000000,4,1,132.081,-6.707,Como un tsunami siento el aire entre mis dedos...,"[C, G, Am, F]","[C, F, G, C]","[0, 7, 21, 5]","[0, 5, 7, 0]"
2,2yptV8t4OH935km6IZWt1l,Cosas Que Suenan A...,Maldita Nerea,"[C, G, Am, F, F, G, C, Am]",0.0350,0.000019,2,1,148.968,-8.005,Yo nado en mares pequeños\nY que todo salga bi...,"[C, G, Am, F]","[F, C, G, Am]","[0, 7, 21, 5]","[5, 0, 7, 21]"


In [147]:
col_chords = ['chords_1_numeric', 'chords_2_numeric']

# Aplicar la función a cada fila
for col in col_chords:
    df_vectorial[f'normal_{col}'] = df_vectorial.apply(lambda row: transpose_chords_to_key(row, col), axis=1)

df_vectorial

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric
0,1uJnb0kS96zFOrjCI97JV8,He Vuelto,El Barrio,"[C, G, Am, F, F, G, C, Am]",0.1760,0.000000,4,1,164.974,-3.762,He vuelto\nSe acabó este descansito\nAhora toc...,"[Am, G, F, E]","[C, G, Am, E]","[21, 7, 5, 4]","[0, 7, 21, 4]","[17, 3, 1, 0]","[-4, 3, 17, 0]"
1,1kG7gfH4HDUlDht0VZYkiP,Libertad,Nil Moliner,"[C, G, Am, F, G, D, Em, C]",0.0303,0.000000,4,1,132.081,-6.707,Como un tsunami siento el aire entre mis dedos...,"[C, G, Am, F]","[C, F, G, C]","[0, 7, 21, 5]","[0, 5, 7, 0]","[-4, 3, 17, 1]","[-4, 1, 3, -4]"
2,2yptV8t4OH935km6IZWt1l,Cosas Que Suenan A...,Maldita Nerea,"[C, G, Am, F, F, G, C, Am]",0.0350,0.000019,2,1,148.968,-8.005,Yo nado en mares pequeños\nY que todo salga bi...,"[C, G, Am, F]","[F, C, G, Am]","[0, 7, 21, 5]","[5, 0, 7, 21]","[-2, 5, 19, 3]","[3, -2, 5, 19]"
3,41JrRedwrJxbBPwI8Qku04,Pacto Entre Caballeros,Joaquín Sabina,"[Am, G, F, E, C, G, Am, F]",0.0601,0.000005,10,1,206.752,-11.716,"(Joaquin Sabina / J. Batanero, P. Varona)\nNo ...","[G, D, Em, C]","[A, E, F#m, D]","[7, 2, 16, 0]","[9, 4, 18, 2]","[-3, -8, 6, -10]","[-1, -6, 8, -8]"
4,4fYAcwdEtW2VCZe5Xoe7iC,A Contracorriente,El Canto Del Loco,"[C, G, Am, F, Em, C, G, D]",0.0390,0.000007,8,1,155.009,-1.945,Todo lo que hacía ese día me salía muy mal\nDe...,"[C, G, Am, F]","[C, G, Am, F]","[0, 7, 21, 5]","[0, 7, 21, 5]","[-8, -1, 13, -3]","[-8, -1, 13, -3]"


In [148]:
for col in col_chords:
    df_vectorial[f'diff_{col}'] = df_vectorial.apply(lambda row: calc_diff(row[col]), axis=1)

df_vectorial

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,1uJnb0kS96zFOrjCI97JV8,He Vuelto,El Barrio,"[C, G, Am, F, F, G, C, Am]",0.1760,0.000000,4,1,164.974,-3.762,He vuelto\nSe acabó este descansito\nAhora toc...,"[Am, G, F, E]","[C, G, Am, E]","[21, 7, 5, 4]","[0, 7, 21, 4]","[17, 3, 1, 0]","[-4, 3, 17, 0]","[-14, -2, -1]","[7, 14, -17]"
1,1kG7gfH4HDUlDht0VZYkiP,Libertad,Nil Moliner,"[C, G, Am, F, G, D, Em, C]",0.0303,0.000000,4,1,132.081,-6.707,Como un tsunami siento el aire entre mis dedos...,"[C, G, Am, F]","[C, F, G, C]","[0, 7, 21, 5]","[0, 5, 7, 0]","[-4, 3, 17, 1]","[-4, 1, 3, -4]","[7, 14, -16]","[5, 2, -7]"
2,2yptV8t4OH935km6IZWt1l,Cosas Que Suenan A...,Maldita Nerea,"[C, G, Am, F, F, G, C, Am]",0.0350,0.000019,2,1,148.968,-8.005,Yo nado en mares pequeños\nY que todo salga bi...,"[C, G, Am, F]","[F, C, G, Am]","[0, 7, 21, 5]","[5, 0, 7, 21]","[-2, 5, 19, 3]","[3, -2, 5, 19]","[7, 14, -16]","[-5, 7, 14]"
3,41JrRedwrJxbBPwI8Qku04,Pacto Entre Caballeros,Joaquín Sabina,"[Am, G, F, E, C, G, Am, F]",0.0601,0.000005,10,1,206.752,-11.716,"(Joaquin Sabina / J. Batanero, P. Varona)\nNo ...","[G, D, Em, C]","[A, E, F#m, D]","[7, 2, 16, 0]","[9, 4, 18, 2]","[-3, -8, 6, -10]","[-1, -6, 8, -8]","[-5, 14, -16]","[-5, 14, -16]"
4,4fYAcwdEtW2VCZe5Xoe7iC,A Contracorriente,El Canto Del Loco,"[C, G, Am, F, Em, C, G, D]",0.0390,0.000007,8,1,155.009,-1.945,Todo lo que hacía ese día me salía muy mal\nDe...,"[C, G, Am, F]","[C, G, Am, F]","[0, 7, 21, 5]","[0, 7, 21, 5]","[-8, -1, 13, -3]","[-8, -1, 13, -3]","[7, 14, -16]","[7, 14, -16]"


In [149]:
df_vectorial.replace('\n', ' ', regex=True, inplace=True)
#elimino los saltos de línea en la letra y temática de la canción

In [150]:
df_vectorial = df_vectorial.fillna(0)

In [151]:
df_vectorial.head(3)

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,1uJnb0kS96zFOrjCI97JV8,He Vuelto,El Barrio,"[C, G, Am, F, F, G, C, Am]",0.1760,0.000000,4,1,164.974,-3.762,He vuelto Se acabó este descansito Ahora toca ...,"[Am, G, F, E]","[C, G, Am, E]","[21, 7, 5, 4]","[0, 7, 21, 4]","[17, 3, 1, 0]","[-4, 3, 17, 0]","[-14, -2, -1]","[7, 14, -17]"
1,1kG7gfH4HDUlDht0VZYkiP,Libertad,Nil Moliner,"[C, G, Am, F, G, D, Em, C]",0.0303,0.000000,4,1,132.081,-6.707,Como un tsunami siento el aire entre mis dedos...,"[C, G, Am, F]","[C, F, G, C]","[0, 7, 21, 5]","[0, 5, 7, 0]","[-4, 3, 17, 1]","[-4, 1, 3, -4]","[7, 14, -16]","[5, 2, -7]"
2,2yptV8t4OH935km6IZWt1l,Cosas Que Suenan A...,Maldita Nerea,"[C, G, Am, F, F, G, C, Am]",0.0350,0.000019,2,1,148.968,-8.005,Yo nado en mares pequeños Y que todo salga bie...,"[C, G, Am, F]","[F, C, G, Am]","[0, 7, 21, 5]","[5, 0, 7, 21]","[-2, 5, 19, 3]","[3, -2, 5, 19]","[7, 14, -16]","[-5, 7, 14]"


In [152]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,42VgiRyg0YjtRVLiPFNxPJ,Zapatillas,El Canto Del Loco,"[C, G, Am, F, C, G, Am, F]","[C, G, Am, F, C, G, F, G]",0.178,0,8,0,198.076,-2.382,Estoy cansado de salir de noche y ver siempre ...,"[C, G, Am, F]","[C, G, Am, F]","[0, 7, 21, 5]","[0, 7, 21, 5]","[-8, -1, 13, -3]","[-8, -1, 13, -3]","[7, 14, -16]","[7, 14, -16]"


# Backup csv

In [153]:
csv_file = 'mi_archivo_temporal.csv'

# Llamar la función para exportar
export_unique_rows(df_vectorial, csv_file)



/Users/gabrielacaseromonge/Documents/GitHub/song-sarter-streamlit/utils/functions.py:277: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_combined = df_combined.applymap(lambda x: str(x) if isinstance(x, list) else x)
/Users/gabrielacaseromonge/Documents/GitHub/song-sarter-streamlit/utils/functions.py:283: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_combined = df_combined.applymap(lambda x: eval(x) if isinstance(x, str) and x.startswith('[') and x.endswith(']') else x)


# Expand columns

In [154]:
# Especificar las columnas que contienen listas (excepto 'embedding')
columns_to_expand = ['instruments', 'chords_1', 'chords_2', 
                      'chords_1_numeric', 'chords_2_numeric', 
                      'normal_chords_1_numeric', 'normal_chords_2_numeric', 
                      'diff_chords_1_numeric', 'diff_chords_2_numeric']

Input:

In [155]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,42VgiRyg0YjtRVLiPFNxPJ,Zapatillas,El Canto Del Loco,"[C, G, Am, F, C, G, Am, F]","[C, G, Am, F, C, G, F, G]",0.178,0,8,0,198.076,-2.382,Estoy cansado de salir de noche y ver siempre ...,"[C, G, Am, F]","[C, G, Am, F]","[0, 7, 21, 5]","[0, 7, 21, 5]","[-8, -1, 13, -3]","[-8, -1, 13, -3]","[7, 14, -16]","[7, 14, -16]"


In [156]:
columns_to_expand

['instruments',
 'chords_1',
 'chords_2',
 'chords_1_numeric',
 'chords_2_numeric',
 'normal_chords_1_numeric',
 'normal_chords_2_numeric',
 'diff_chords_1_numeric',
 'diff_chords_2_numeric']

In [157]:
input_expanded = expand_and_remove_original_columns(input_data, columns_to_expand)

In [158]:
input_expanded

,id,name,artists,chords,speechiness,instrumentalness,key,mode,tempo,loudness,...,normal_chords_2_numeric_1,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3
0,42VgiRyg0YjtRVLiPFNxPJ,Zapatillas,El Canto Del Loco,"[C, G, Am, F, C, G, Am, F]",0.178,0,8,0,198.076,-2.382,...,-8,-1,13,-3,7,14,-16,7,14,-16


In [159]:
#elimino las columnas que tengan solo valores nulos
input_expanded = input_expanded.dropna(axis=1, how='all')

Related tracks:

In [160]:
df_vectorial_expanded = expand_and_remove_original_columns(df_vectorial, columns_to_expand)

In [161]:
df_vectorial_expanded

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_1,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3
0,1uJnb0kS96zFOrjCI97JV8,He Vuelto,El Barrio,0.1760,0.000000,4,1,164.974,-3.762,He vuelto Se acabó este descansito Ahora toca ...,...,-4,3,17,0,-14,-2,-1,7,14,-17
1,1kG7gfH4HDUlDht0VZYkiP,Libertad,Nil Moliner,0.0303,0.000000,4,1,132.081,-6.707,Como un tsunami siento el aire entre mis dedos...,...,-4,1,3,-4,7,14,-16,5,2,-7
2,2yptV8t4OH935km6IZWt1l,Cosas Que Suenan A...,Maldita Nerea,0.0350,0.000019,2,1,148.968,-8.005,Yo nado en mares pequeños Y que todo salga bie...,...,3,-2,5,19,7,14,-16,-5,7,14
3,41JrRedwrJxbBPwI8Qku04,Pacto Entre Caballeros,Joaquín Sabina,0.0601,0.000005,10,1,206.752,-11.716,"(Joaquin Sabina / J. Batanero, P. Varona) No p...",...,-1,-6,8,-8,-5,14,-16,-5,14,-16
4,4fYAcwdEtW2VCZe5Xoe7iC,A Contracorriente,El Canto Del Loco,0.0390,0.000007,8,1,155.009,-1.945,Todo lo que hacía ese día me salía muy mal Des...,...,-8,-1,13,-3,7,14,-16,7,14,-16


In [162]:
#elimino las columnas que tengan solo valores nulos
df_vectorial_expanded = df_vectorial_expanded.dropna(axis=1, how='all')

In [163]:
drop_columns_with_many_nulls(df_vectorial_expanded, 15)

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_1,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3
0,1uJnb0kS96zFOrjCI97JV8,He Vuelto,El Barrio,0.1760,0.000000,4,1,164.974,-3.762,He vuelto Se acabó este descansito Ahora toca ...,...,-4,3,17,0,-14,-2,-1,7,14,-17
1,1kG7gfH4HDUlDht0VZYkiP,Libertad,Nil Moliner,0.0303,0.000000,4,1,132.081,-6.707,Como un tsunami siento el aire entre mis dedos...,...,-4,1,3,-4,7,14,-16,5,2,-7
2,2yptV8t4OH935km6IZWt1l,Cosas Que Suenan A...,Maldita Nerea,0.0350,0.000019,2,1,148.968,-8.005,Yo nado en mares pequeños Y que todo salga bie...,...,3,-2,5,19,7,14,-16,-5,7,14
3,41JrRedwrJxbBPwI8Qku04,Pacto Entre Caballeros,Joaquín Sabina,0.0601,0.000005,10,1,206.752,-11.716,"(Joaquin Sabina / J. Batanero, P. Varona) No p...",...,-1,-6,8,-8,-5,14,-16,-5,14,-16
4,4fYAcwdEtW2VCZe5Xoe7iC,A Contracorriente,El Canto Del Loco,0.0390,0.000007,8,1,155.009,-1.945,Todo lo que hacía ese día me salía muy mal Des...,...,-8,-1,13,-3,7,14,-16,7,14,-16


In [164]:
df_vectorial_expanded.head(3)

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_1,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3
0,1uJnb0kS96zFOrjCI97JV8,He Vuelto,El Barrio,0.1760,0.000000,4,1,164.974,-3.762,He vuelto Se acabó este descansito Ahora toca ...,...,-4,3,17,0,-14,-2,-1,7,14,-17
1,1kG7gfH4HDUlDht0VZYkiP,Libertad,Nil Moliner,0.0303,0.000000,4,1,132.081,-6.707,Como un tsunami siento el aire entre mis dedos...,...,-4,1,3,-4,7,14,-16,5,2,-7
2,2yptV8t4OH935km6IZWt1l,Cosas Que Suenan A...,Maldita Nerea,0.0350,0.000019,2,1,148.968,-8.005,Yo nado en mares pequeños Y que todo salga bie...,...,3,-2,5,19,7,14,-16,-5,7,14


# Create column for the text to create the embeddings

Input:

In [165]:
input_expanded=to_embbed_text_column(input_expanded)

In [166]:
input_expanded['to_embbed_text']

0    name: Zapatillas artists: El Canto Del Loco ch...
Name: to_embbed_text, dtype: object

Related songs:

In [167]:
df_vectorial_expanded=to_embbed_text_column(df_vectorial_expanded)

In [168]:
df_vectorial_expanded


,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3,to_embbed_text
0,1uJnb0kS96zFOrjCI97JV8,He Vuelto,El Barrio,0.1760,0.000000,4,1,164.974,-3.762,He vuelto Se acabó este descansito Ahora toca ...,...,3,17,0,-14,-2,-1,7,14,-17,name: He Vuelto artists: El Barrio speechiness...
1,1kG7gfH4HDUlDht0VZYkiP,Libertad,Nil Moliner,0.0303,0.000000,4,1,132.081,-6.707,Como un tsunami siento el aire entre mis dedos...,...,1,3,-4,7,14,-16,5,2,-7,name: Libertad artists: Nil Moliner speechines...
2,2yptV8t4OH935km6IZWt1l,Cosas Que Suenan A...,Maldita Nerea,0.0350,0.000019,2,1,148.968,-8.005,Yo nado en mares pequeños Y que todo salga bie...,...,-2,5,19,7,14,-16,-5,7,14,name: Cosas Que Suenan A... artists: Maldita N...
3,41JrRedwrJxbBPwI8Qku04,Pacto Entre Caballeros,Joaquín Sabina,0.0601,0.000005,10,1,206.752,-11.716,"(Joaquin Sabina / J. Batanero, P. Varona) No p...",...,-6,8,-8,-5,14,-16,-5,14,-16,name: Pacto Entre Caballeros artists: Joaquín ...
4,4fYAcwdEtW2VCZe5Xoe7iC,A Contracorriente,El Canto Del Loco,0.0390,0.000007,8,1,155.009,-1.945,Todo lo que hacía ese día me salía muy mal Des...,...,-1,13,-3,7,14,-16,7,14,-16,name: A Contracorriente artists: El Canto Del ...


# Text for embeddings to chat gpt

In [169]:
api_key = os.getenv("OPENAI_API_KEY")

# Configurar la clave de API en OpenAI
OpenAI.api_key = api_key

In [170]:
# Convert text into vectors using embeddings
client = OpenAI(
 api_key=api_key,
)

Input:

In [171]:
input_expanded['embedding'] = input_expanded['to_embbed_text'].apply(lambda x: client.embeddings.create(
model="text-embedding-3-small",
input=x
).data[0].embedding)

In [172]:
input_expanded.drop(columns=['to_embbed_text'], inplace=True)

In [173]:
df_vectorial_expanded

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3,to_embbed_text
0,1uJnb0kS96zFOrjCI97JV8,He Vuelto,El Barrio,0.1760,0.000000,4,1,164.974,-3.762,He vuelto Se acabó este descansito Ahora toca ...,...,3,17,0,-14,-2,-1,7,14,-17,name: He Vuelto artists: El Barrio speechiness...
1,1kG7gfH4HDUlDht0VZYkiP,Libertad,Nil Moliner,0.0303,0.000000,4,1,132.081,-6.707,Como un tsunami siento el aire entre mis dedos...,...,1,3,-4,7,14,-16,5,2,-7,name: Libertad artists: Nil Moliner speechines...
2,2yptV8t4OH935km6IZWt1l,Cosas Que Suenan A...,Maldita Nerea,0.0350,0.000019,2,1,148.968,-8.005,Yo nado en mares pequeños Y que todo salga bie...,...,-2,5,19,7,14,-16,-5,7,14,name: Cosas Que Suenan A... artists: Maldita N...
3,41JrRedwrJxbBPwI8Qku04,Pacto Entre Caballeros,Joaquín Sabina,0.0601,0.000005,10,1,206.752,-11.716,"(Joaquin Sabina / J. Batanero, P. Varona) No p...",...,-6,8,-8,-5,14,-16,-5,14,-16,name: Pacto Entre Caballeros artists: Joaquín ...
4,4fYAcwdEtW2VCZe5Xoe7iC,A Contracorriente,El Canto Del Loco,0.0390,0.000007,8,1,155.009,-1.945,Todo lo que hacía ese día me salía muy mal Des...,...,-1,13,-3,7,14,-16,7,14,-16,name: A Contracorriente artists: El Canto Del ...


Related songs:

In [174]:
#para cada fila del dataframe, sacar el vector de embedding de la columna "to_embbed_text"
df_vectorial_expanded['embedding'] = df_vectorial_expanded['to_embbed_text'].apply(lambda x: client.embeddings.create(
model="text-embedding-3-small",
input=x
).data[0].embedding)

In [175]:
df_vectorial_expanded.drop(columns=['to_embbed_text'], inplace=True)

In [176]:
df_vectorial_expanded = df_vectorial_expanded.rename(columns={'embedding': 'values'})

# Data to pinecone

In [177]:

# Procesar el DataFrame
df_vectorial_expanded = create_metadata_column(df_vectorial_expanded)
df_vectorial_expanded = clean_metadata_column(df_vectorial_expanded)
df_para_pinecone = create_dataframe_for_pinecone(df_vectorial_expanded)
insert_into_pinecone(df_para_pinecone, index)



sending upsert requests:   0%|          | 0/5 [00:00<?, ?it/s]

In [178]:
df_vectorial_expanded

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3,values,metadata
0,1uJnb0kS96zFOrjCI97JV8,He Vuelto,El Barrio,0.1760,0.000000,4,1,164.974,-3.762,He vuelto Se acabó este descansito Ahora toca ...,...,17,0,-14,-2,-1,7,14,-17,"[0.014554610475897789, -0.003263365477323532, ...","{'name': 'He Vuelto', 'artists': 'El Barrio', ..."
1,1kG7gfH4HDUlDht0VZYkiP,Libertad,Nil Moliner,0.0303,0.000000,4,1,132.081,-6.707,Como un tsunami siento el aire entre mis dedos...,...,3,-4,7,14,-16,5,2,-7,"[0.030190657824277878, 0.010753573849797249, 0...","{'name': 'Libertad', 'artists': 'Nil Moliner',..."
2,2yptV8t4OH935km6IZWt1l,Cosas Que Suenan A...,Maldita Nerea,0.0350,0.000019,2,1,148.968,-8.005,Yo nado en mares pequeños Y que todo salga bie...,...,5,19,7,14,-16,-5,7,14,"[0.027277356013655663, 0.0350077822804451, 0.0...","{'name': 'Cosas Que Suenan A...', 'artists': '..."
3,41JrRedwrJxbBPwI8Qku04,Pacto Entre Caballeros,Joaquín Sabina,0.0601,0.000005,10,1,206.752,-11.716,"(Joaquin Sabina / J. Batanero, P. Varona) No p...",...,8,-8,-5,14,-16,-5,14,-16,"[0.0008626123308204114, 0.01796708069741726, -...","{'name': 'Pacto Entre Caballeros', 'artists': ..."
4,4fYAcwdEtW2VCZe5Xoe7iC,A Contracorriente,El Canto Del Loco,0.0390,0.000007,8,1,155.009,-1.945,Todo lo que hacía ese día me salía muy mal Des...,...,13,-3,7,14,-16,7,14,-16,"[0.01818690076470375, 0.011262848041951656, -0...","{'name': 'A Contracorriente', 'artists': 'El C..."


In [179]:
df_para_pinecone

,id,values,metadata
0,1uJnb0kS96zFOrjCI97JV8,"[0.014554610475897789, -0.003263365477323532, ...","{'name': 'He Vuelto', 'artists': 'El Barrio', ..."
1,1kG7gfH4HDUlDht0VZYkiP,"[0.030190657824277878, 0.010753573849797249, 0...","{'name': 'Libertad', 'artists': 'Nil Moliner',..."
2,2yptV8t4OH935km6IZWt1l,"[0.027277356013655663, 0.0350077822804451, 0.0...","{'name': 'Cosas Que Suenan A...', 'artists': '..."
3,41JrRedwrJxbBPwI8Qku04,"[0.0008626123308204114, 0.01796708069741726, -...","{'name': 'Pacto Entre Caballeros', 'artists': ..."
4,4fYAcwdEtW2VCZe5Xoe7iC,"[0.01818690076470375, 0.011262848041951656, -0...","{'name': 'A Contracorriente', 'artists': 'El C..."


# Query search in pinecone

In [180]:
response = realizar_consulta(client, input_expanded, index, top_k=5)

Resultado de la consulta: {'matches': [{'id': '1rqGuE3jvcvacQk96lfnl6',
              'metadata': {'artists': 'Geese',
                           'chords_1_1': 'G',
                           'chords_1_2': 'D',
                           'chords_1_3': 'Em',
                           'chords_1_4': 'C',
                           'chords_1_numeric_1': 7.0,
                           'chords_1_numeric_2': 2.0,
                           'chords_1_numeric_3': 16.0,
                           'chords_1_numeric_4': 0.0,
                           'chords_2_1': 'Am',
                           'chords_2_2': 'F',
                           'chords_2_3': 'C',
                           'chords_2_4': 'G',
                           'chords_2_numeric_1': 21.0,
                           'chords_2_numeric_2': 5.0,
                           'chords_2_numeric_3': 0.0,
                           'chords_2_numeric_4': 7.0,
                           'diff_chords_1_numeric_1': -5.0,
                 

In [181]:
response_matches = response['matches']

details = []
for match in response_matches:
    details.append(f"Intruments: {match['metadata']['instruments_1']} {match['metadata']['instruments_2']}, Tempo: {match['metadata']['tempo']}, Speechiness: {match['metadata']['speechiness']}, Loudness: {match['metadata']['loudness']}")

In [182]:
details

['Intruments: guitar bass, Tempo: 131.948, Speechiness: 0.0382, Loudness: -7.699',
 'Intruments: piano cello, Tempo: 133.355, Speechiness: 0.033, Loudness: -12.636',
 'Intruments: guitar saxophone, Tempo: 131.685, Speechiness: 0.0469, Loudness: -4.09',
 'Intruments: piano drums, Tempo: 124.357, Speechiness: 0.0678, Loudness: -7.621',
 'Intruments: piano guitar, Tempo: 114.032, Speechiness: 0.0388, Loudness: -6.153']

In [183]:

for match in response_matches:
    print(f"Letra: {match['metadata']['theme']}, \n Score: {match['score']}")

Letra: I’ve been hit by the bus of love That falling brick of you Baby, when there’s tears in your eyes You don’t have to lie, 
 Score: 0.214397848
Letra: Love and learn Take your turn Somewhere there's a feather Falling slowly from the sky, 
 Score: 0.206153154
Letra: The song "El Encuentro" by Biela explores themes of connection, longing, and intimacy. It captures the emotions surrounding a significant meeting between two individuals, emphasizing the beauty and complexity of human relationships. Through its lyrical imagery, it conveys a sense of urgency and the profound impact such encounters can have on one’s life. Overall, the song reflects on the joys and challenges of love and the deep bonds formed through shared experiences., 
 Score: 0.203557625
Letra: Oh-oh-oh-oh Mhmm, mhmm From Bamenda, it's Libianca Oh-oh, 
 Score: 0.198177159
Letra: The theme of "Miles and Miles of Texas" revolves around the deep connection to the vast Texas landscape and the sense of freedom it brings to t

In [184]:
diff_chord_2= input_data['diff_chords_2_numeric'].iloc[0]
diff_chord_1= input_data['diff_chords_1_numeric'].iloc[0]

In [185]:
# Crear el embedding de la nueva query
chords_query = f"A song with a similar progession and distance (diff) in chords to {input_data['chords_1'][0]},y diff chords {diff_chord_2},{diff_chord_1}."
theme_embedding = client.embeddings.create(
    model="text-embedding-3-small",
    input=chords_query
).data[0].embedding

chords_query = index.query(vector=theme_embedding, top_k=5, include_metadata=True)

chords_query_ = chords_query['matches']



In [186]:
for match in chords_query_:
    print(f"Chords: {match['metadata']['chords_1_1']} {match['metadata']['chords_1_2']} {match['metadata']['chords_1_3']} {match['metadata']['chords_1_4']}, {match['metadata']['chords_2_1']} {match['metadata']['chords_2_2']} {match['metadata']['chords_2_3']} {match['metadata']['chords_2_4']}, \n Score: {match['score']}")

Chords: C G Am F, F Am C G, 
 Score: 0.465003759
Chords: C G Am F, C F G Am, 
 Score: 0.45702365
Chords: C G Am F, C G F F, 
 Score: 0.452040702
Chords: 2023 Remaster C G Am, F C G F, 
 Score: 0.451456219
Chords: C G Am F, D A Bm G, 
 Score: 0.448484957


In [187]:
chord_wheels=[]
for match in chords_query_:
    chord_wheels.append(f"Chord wheel: {match['metadata']['chords_1_1']} {match['metadata']['chords_1_2']} {match['metadata']['chords_1_3']} {match['metadata']['chords_1_4']}, {match['metadata']['chords_2_1']} {match['metadata']['chords_2_2']} {match['metadata']['chords_2_3']} {match['metadata']['chords_2_4']}")

In [188]:
chord_wheels

['Chord wheel: C G Am F, F Am C G',
 'Chord wheel: C G Am F, C F G Am',
 'Chord wheel: C G Am F, C G F F',
 'Chord wheel: 2023 Remaster C G Am, F C G F',
 'Chord wheel: C G Am F, D A Bm G']

In [189]:
input_data['theme'][0]

'Estoy cansado de salir de noche y ver siempre la misma gente. Estoy flipando de que la gente se invente, cuente y luego reinvente. Apotronado en el sofá de mi casa, vente se está caliente. Amaestrados vamos al mismo sitio todos aunque luego ni entres. Alucinando de que me miren de arriba a bajo como un delincuente. Intoxicado de que me pongan esa puta música indiferente.  (ESTRIBILLO)'

In [190]:
# Crear el embedding de la nueva query
lyrics_query = f"Give me a similar theme song to {input_data['theme'][0]}"
theme_embedding = client.embeddings.create(
    model="text-embedding-3-small",
    input=lyrics_query
).data[0].embedding

lyrics_result = index.query(vector=theme_embedding, top_k=5, include_metadata=True)

lyrics_matches = lyrics_result['matches']



In [191]:
lyrics_list = []  # Crear una lista vacía para almacenar los textos

for match in lyrics_matches:
    # Agregar cada texto generado a la lista
    lyrics_list.append(f"Lyrics: {match['metadata']['theme']}")



for match in lyrics_matches:
    # Agregar cada texto generado a la lista
    print(f"Lyrics: {match['metadata']['theme']} {match['id']} Score: {match['score']}")

lyrics_list.append(input_data['theme'][0])

Lyrics: Estoy cansado de salir de noche y ver siempre la misma gente. Estoy flipando de que la gente se invente, cuente y luego reinvente. Apotronado en el sofá de mi casa, vente se está caliente. Amaestrados vamos al mismo sitio todos aunque luego ni entres. Alucinando de que me miren de arriba a bajo como un delincuente. Intoxicado de que me pongan esa puta música indiferente.  (ESTRIBILLO) 42VgiRyg0YjtRVLiPFNxPJ Score: 0.65762794
Lyrics: Dime, que la noche aqui no acaba que me quedo como estaba con las ganas de volar. Dime, que me subo por las ramas la sangre se me derrama  y estoy arto de aguantar.  7p86AB1S0CeNeYc7CfWvGR Score: 0.539999
Lyrics: Dónde estabas entonces cuando tanto te necesité? Nadie es mejor que nadie pero tu creíste vencer. Si lloré ante tu puerta de nada sirvió. Barras de bar, vertederos de amor... Os enseñé mi trocito peor.  Retales de mi vida,  6xnHAQ96sirlw7Gm4ppqxG Score: 0.527094543
Lyrics: Porque este mundo no lo entiendo,  Porque hay verano y hay invierno,

In [192]:
lyrics_list

['Lyrics: Estoy cansado de salir de noche y ver siempre la misma gente. Estoy flipando de que la gente se invente, cuente y luego reinvente. Apotronado en el sofá de mi casa, vente se está caliente. Amaestrados vamos al mismo sitio todos aunque luego ni entres. Alucinando de que me miren de arriba a bajo como un delincuente. Intoxicado de que me pongan esa puta música indiferente.  (ESTRIBILLO)',
 'Lyrics: Dime, que la noche aqui no acaba que me quedo como estaba con las ganas de volar. Dime, que me subo por las ramas la sangre se me derrama  y estoy arto de aguantar. ',
 'Lyrics: Dónde estabas entonces cuando tanto te necesité? Nadie es mejor que nadie pero tu creíste vencer. Si lloré ante tu puerta de nada sirvió. Barras de bar, vertederos de amor... Os enseñé mi trocito peor.  Retales de mi vida, ',
 'Lyrics: Porque este mundo no lo entiendo,  Porque hay verano y hay invierno,  Hay alegría y dolor,  Hay una cara y su cruz  Nos conocimos en Enero  Y me olvidaste en Febrero   Y ahora 

In [193]:
nueva_cancion = create_song(lyrics_list, chord_wheels, details)
print(nueva_cancion)

# Mundo Repetido

**Tempo:** 131 BPM  
**Instrumentos principales:** Guitarra, Bajo

### Letra y acordes:

1. **Verso 1**  
   [C] Salgo a la calle, [G] veo el mismo lugar,  
   [Am] Rostros de plástico, [F] vuelven a chocar.  
   [C] Cansado de cuentos, [G] que nadie escuchará,  
   [Am] En casa me quedo, [F] nadie me encontrará.

2. **Coro**  
   [F] Dime por qué, [Am] este ciclo no termina,  
   [C] Marchando al mismo [G] compás en la rutina.  
   [F] Quiero volar, [Am] romper esta cadena,  
   [C] La música que [G] suena, siempre tan ajena.

3. **Verso 2**  
   [C] Miradas que juzgan, [G] sin conocerme,  
   [Am] Ideas que reinventan, [F] sin entenderme.  
   [C] El mundo gira, [G] en esta monotonía,  
   [Am] Busco escaparme, [F] sin tanta hipocresía.

4. **Coro final**  
   [F] Dime por qué, [Am] el reloj avanza lento,  
   [C] Caminamos en [G] círculos como el viento.  
   [F] Quiero volar, [Am] sentir la libertad,  
   [C] La música se [G] apagará en la soledad. 

---

**Nota**

In [194]:
nueva_cancion

'# Mundo Repetido\n\n**Tempo:** 131 BPM  \n**Instrumentos principales:** Guitarra, Bajo\n\n### Letra y acordes:\n\n1. **Verso 1**  \n   [C] Salgo a la calle, [G] veo el mismo lugar,  \n   [Am] Rostros de plástico, [F] vuelven a chocar.  \n   [C] Cansado de cuentos, [G] que nadie escuchará,  \n   [Am] En casa me quedo, [F] nadie me encontrará.\n\n2. **Coro**  \n   [F] Dime por qué, [Am] este ciclo no termina,  \n   [C] Marchando al mismo [G] compás en la rutina.  \n   [F] Quiero volar, [Am] romper esta cadena,  \n   [C] La música que [G] suena, siempre tan ajena.\n\n3. **Verso 2**  \n   [C] Miradas que juzgan, [G] sin conocerme,  \n   [Am] Ideas que reinventan, [F] sin entenderme.  \n   [C] El mundo gira, [G] en esta monotonía,  \n   [Am] Busco escaparme, [F] sin tanta hipocresía.\n\n4. **Coro final**  \n   [F] Dime por qué, [Am] el reloj avanza lento,  \n   [C] Caminamos en [G] círculos como el viento.  \n   [F] Quiero volar, [Am] sentir la libertad,  \n   [C] La música se [G] apagará 